Messing around with the idea of creating a formal parser for the IR.

But perhaps it would be better to just output code that's a subset of Python, so you can use the native Python parser?

In [2]:
import lark

In [16]:
formal_grammar = r"""
program       ::= assignment+
assignment    ::= VAR "=" function_call NEWLINE
function_call ::= VAR "(" [ arg_list ] ")"
arg_list      ::= VAR ("," VAR)*

(* Terminal Definitions *)
VAR           ::= [a-zA-Z_][a-zA-Z0-9_]*
NEWLINE       ::= "\n"
"""

lark_grammar = r"""
    ?program: assignment+

    ?assignment: VAR "=" function_call NEWLINE

    ?function_call: VAR "(" [arg_list] ")"

    ?arg_list: VAR ("," VAR)*

    %import common.CNAME -> VAR
    %import common.WS
    %import common.NEWLINE
    %ignore WS
"""

parser = lark.Lark(lark_grammar, start='program')

source = """
a = f()
b = g(a)
c = f()
d = h(b, c)
e = another_func(a, b, c, d)
"""

tree = parser.parse(source)

print(tree.pretty())

program
  assignment
    a
    function_call
      f
      None
    

  assignment
    b
    function_call
      g
      a
    

  assignment
    c
    function_call
      f
      None
    

  assignment
    d
    function_call
      h
      arg_list
        b
        c
    

  assignment
    e
    function_call
      another_func
      arg_list
        a
        b
        c
        d
    




In [18]:
# let's create a grammar that has "nodes" and "packages"

lark_grammar = r"""
    ?program: statement+
    ?statement: (node_assignment | package_assignment) NEWLINE

    node_assignment: VAR "=" node_creation
    node_creation: "node" "(" package_expression ("," VAR)* ")"

    package_assignment: VAR "=" package_creation
    package_expression: package_creation | VAR
    package_creation: VAR "(" ")"

    %import common.CNAME -> VAR
    %import common.NEWLINE
    %import common.WS
    %ignore WS
"""

source = r"""
a = package_fun1()
x = node(a)
"""

parser = lark.Lark(lark_grammar, start='program')

tree = parser.parse(source)

print(tree.pretty())

program
  statement
    package_assignment
      a
      package_creation	package_fun1
    

  statement
    node_assignment
      x
      node_creation
        package_expression	a
    




In [19]:
source = r"""
x = node(package_fun1())
"""

parser = lark.Lark(lark_grammar, start='program')

tree = parser.parse(source)

print(tree.pretty())

statement
  node_assignment
    x
    node_creation
      package_expression
        package_creation	package_fun1
  


